In [2]:
caffe_root = '../'  # this file should be run from {caffe_root}/examples (otherwise change this line)

import sys
sys.path.insert(0, caffe_root + 'python')
import caffe

caffe.set_device(0)
caffe.set_mode_gpu()

import numpy as np
from pylab import *
%matplotlib inline
import tempfile

# Helper function for deprocessing preprocessed images, e.g., for display.
def deprocess_net_image(image):
    image = image.copy()              # don't modify destructively
    image = image[::-1]               # BGR -> RGB
    image = image.transpose(1, 2, 0)  # CHW -> HWC
    image += [123, 117, 104]          # (approximately) undo mean subtraction

    # clamp values in [0, 255]
    image[image < 0], image[image > 255] = 0, 255

    # round and cast from float32 to uint8
    image = np.round(image)
    image = np.require(image, dtype=np.uint8)

    return image

In [3]:
import os
# os.chdir('examples')
weights = os.path.join(caffe_root, 'models/VGG_ILSVRC_16_layers/VGG_ILSVRC_16_layers.caffemodel')
assert os.path.exists(weights)

In [4]:
full_dataset = False
if full_dataset:
    NUM_STYLE_IMAGES = NUM_STYLE_LABELS = -1
else:
    NUM_STYLE_IMAGES =2005
    NUM_STYLE_LABELS = 30
# Load ImageNet labels to imagenet_labels
# imagenet_label_file = caffe_root + 'data/ilsvrc12/synset_words.txt'
# imagenet_labels = list(np.loadtxt(imagenet_label_file, str, delimiter='\t'))
# assert len(imagenet_labels) == 1000
# print 'Loaded ImageNet labels:\n', '\n'.join(imagenet_labels[:10] + ['...'])

# Load style labels to style_labels
style_label_file = caffe_root + 'data/CUB2011/classes.txt'
t=np.loadtxt(style_label_file, str, delimiter='\t')
style_labels =[t[i].split('.')[1] for i in range(200)]
# style_labels =list(np.loadtxt(style_label_file, str, delimiter='\t'))
# if NUM_STYLE_LABELS > 0:
#     style_labels = style_labels[:NUM_STYLE_LABELS]
# print '\nLoaded style labels:\n',style_labels

In [5]:
t=np.loadtxt(style_label_file, str, delimiter='\t')

In [17]:
t.shape

(200,)

In [5]:
from caffe import layers as L
from caffe import params as P

weight_param = dict(lr_mult=1, decay_mult=1)
bias_param   = dict(lr_mult=2, decay_mult=0)
learned_param = [weight_param, bias_param]

frozen_param = [dict(lr_mult=0)] * 2

def conv_relu(bottom, ks, nout, stride=1, pad=0, group=1,
              param=learned_param,
              weight_filler=dict(type='gaussian', std=0.01),
              bias_filler=dict(type='constant', value=0.1)):
    conv = L.Convolution(bottom, kernel_size=ks, stride=stride,
                         num_output=nout, pad=pad, group=group,
                         param=param, weight_filler=weight_filler,
                         bias_filler=bias_filler)
    return conv, L.ReLU(conv, in_place=True)

def fc_relu(bottom, nout, param=learned_param,
            weight_filler=dict(type='gaussian', std=0.005),
            bias_filler=dict(type='constant', value=0.1)):
    fc = L.InnerProduct(bottom, num_output=nout, param=param,
                        weight_filler=weight_filler,
                        bias_filler=bias_filler)
    return fc, L.ReLU(fc, in_place=True)

def max_pool(bottom, ks, stride=1):
    return L.Pooling(bottom, pool=P.Pooling.MAX, kernel_size=ks, stride=stride)

def caffenet(data, label=None, train=True, num_classes=1000,
             classifier_name='fc8', learn_all=False):
    """Returns a NetSpec specifying CaffeNet, following the original proto text
       specification (./models/bvlc_reference_caffenet/train_val.prototxt)."""
    n = caffe.NetSpec()
    n.data = data
    param = learned_param if learn_all else frozen_param
    n.conv1_1, n.relu1_1 = conv_relu(n.data,ks=3, nout=64, pad=1,param=param)
    n.conv1_2, n.relu1_2 = conv_relu(n.relu1_1, ks=3 ,nout=64, pad=1,param=param)
    n.pool1 = max_pool(n.relu1_2, ks=2, stride=2)
    n.conv2_1, n.relu2_1 = conv_relu(n.pool1,ks=3,  nout=128, pad=1, param=param)
    n.conv2_2, n.relu2_2 = conv_relu(n.relu2_1, ks=3, nout=128, pad=1,param=param)
    n.pool2 = max_pool(n.relu2_2, ks=2, stride=2)
    n.conv3_1, n.relu3_1 = conv_relu(n.pool2, ks=3, nout=256, pad=1,param=param)
    n.conv3_2, n.relu3_2 = conv_relu(n.relu3_1, ks=3, nout=256, pad=1,param=param)
    n.conv3_3, n.relu3_3 = conv_relu(n.relu3_2, ks=3, nout=256, pad=1,param=param)
    n.pool3 = max_pool(n.relu3_3, ks=2, stride=2)
    n.conv4_1, n.relu4_1 = conv_relu(n.pool3,  ks=3, nout=512, pad=1,param=param)
    n.conv4_2, n.relu4_2 = conv_relu(n.relu4_1, ks=3, nout=512, pad=1,param=param)
    n.conv4_3, n.relu4_3 = conv_relu(n.relu4_2, ks=3, nout=512, pad=1,param=param)
    n.pool4 = max_pool(n.relu4_3, ks=2, stride=2)
    n.conv5_1, n.relu5_1 = conv_relu(n.pool4, ks=3, nout=512, pad=1,param=param)
    n.conv5_2, n.relu5_2 = conv_relu(n.relu5_1, ks=3, nout=512, pad=1,param=param)
    n.conv5_3, n.relu5_3 = conv_relu(n.relu5_2, ks=3, nout=512, pad=1,param=param)
    n.pool5 = max_pool(n.relu5_3, ks=2, stride=2)  
    n.fc6, n.relu6 = fc_relu(n.pool5, 4096, param=param)
    if train:
        n.drop6 = fc7input = L.Dropout(n.relu6, in_place=True)
    else:
        fc7input = n.relu6
    n.fc7, n.relu7 = fc_relu(fc7input, 4096, param=param)
    if train:
        n.drop7 = fc8input = L.Dropout(n.relu7, in_place=True)
    else:
        fc8input = n.relu7
    # always learn fc8 (param=learned_param)
    fc8 = L.InnerProduct(fc8input, num_output=num_classes, param=learned_param)
    # give fc8 the name specified by argument `classifier_name`
    n.__setattr__(classifier_name, fc8)
    if not train:
        n.probs = L.Softmax(fc8)
    if label is not None:
        n.label = label
        n.loss = L.SoftmaxWithLoss(fc8, n.label)
        n.acc = L.Accuracy(fc8, n.label)
    # write the net to a temporary file and return its filename
    with tempfile.NamedTemporaryFile(delete=False) as f:
        f.write(str(n.to_proto()))
        return f.name

In [24]:
# dummy_data = L.DummyData(shape=dict(dim=[1, 3, 224, 224]))
# imagenet_net_filename = caffenet(data=dummy_data, train=False)
# imagenet_net = caffe.Net(imagenet_net_filename, weights, caffe.TEST)
# imagenet_net_filename

In [1]:
def style_net(train=True, learn_all=False, subset=None):
    if subset is None:
        subset = 'train' if train else 'test'
    source = caffe_root + 'data/bird_recognition/%s.txt' % subset
    transform_param = dict(mirror=train, crop_size=224,
        mean_file=caffe_root + 'data/ilsvrc12/imagenet_mean.binaryproto')
    style_data, style_label = L.ImageData(
        transform_param=transform_param, source=source,
        batch_size=50, new_height=256, new_width=256, ntop=2)
    return caffenet(data=style_data, label=style_label, train=train,
                    num_classes=NUM_STYLE_LABELS,
                    classifier_name='fc8_birdrecog',
                    learn_all=learn_all)

In [26]:
# untrained_style_net = caffe.Net(style_net(train=False, subset='train'),
#                                 weights, caffe.TEST)

# untrained_style_net.forward()
# style_data_batch = untrained_style_net.blobs['data'].data.copy()
# style_label_batch = np.array(untrained_style_net.blobs['label'].data, dtype=np.int32)

In [6]:
def disp_preds(net, image, labels, k=5, name='ImageNet'):
    input_blob = net.blobs['data']
    net.blobs['data'].data[0, ...] = image
    probs = net.forward(start='conv1')['probs'][0]
    top_k = (-probs).argsort()[:k]
    print 'top %d predicted %s labels =' % (k, name)
    print '\n'.join('\t(%d) %5.2f%% %s' % (i+1, 100*probs[p], labels[p])
                    for i, p in enumerate(top_k))

def disp_imagenet_preds(net, image):
    disp_preds(net, image, imagenet_labels, name='VGG16Net')

def disp_style_preds(net, image):
    disp_preds(net, image, style_labels, name='style')

In [9]:
# batch_index = 10
# image = style_data_batch[batch_index]
# plt.imshow(deprocess_net_image(image))
# print 'actual label =', style_labels[style_label_batch[batch_index]]

In [10]:
# disp_imagenet_preds(imagenet_net, image)

In [11]:
# disp_style_preds(untrained_style_net, image)

In [12]:
# diff = untrained_style_net.blobs['fc7'].data[0] - imagenet_net.blobs['fc7'].data[0]
# error = (diff ** 2).sum()
# assert error < 1e-8

In [13]:
# del untrained_style_net

In [7]:
from caffe.proto import caffe_pb2

def solver(train_net_path, test_net_path=None, base_lr=0.001):
    s = caffe_pb2.SolverParameter()

    # Specify locations of the train and (maybe) test networks.
    s.train_net = train_net_path
    if test_net_path is not None:
        s.test_net.append(test_net_path)
        s.test_interval = 1000  # Test after every 1000 training iterations.
        s.test_iter.append(100) # Test on 100 batches each time we test.

    # The number of iterations over which to average the gradient.
    # Effectively boosts the training batch size by the given factor, without
    # affecting memory utilization.
    s.iter_size = 1
    
    s.max_iter = 100000     # # of times to update the net (training iterations)
    
    # Solve using the stochastic gradient descent (SGD) algorithm.
    # Other choices include 'Adam' and 'RMSProp'.
    s.type = 'SGD'

    # Set the initial learning rate for SGD.
    s.base_lr = base_lr

    # Set `lr_policy` to define how the learning rate changes during training.
    # Here, we 'step' the learning rate by multiplying it by a factor `gamma`
    # every `stepsize` iterations.
    s.lr_policy = 'step'
    s.gamma = 0.1
    s.stepsize = 20000

    # Set other SGD hyperparameters. Setting a non-zero `momentum` takes a
    # weighted average of the current gradient and previous gradients to make
    # learning more stable. L2 weight decay regularizes learning, to help prevent
    # the model from overfitting.
    s.momentum = 0.9
    s.weight_decay = 5e-4

    # Display the current training loss and accuracy every 1000 iterations.
    s.display = 1000

    # Snapshots are files used to store networks we've trained.  Here, we'll
    # snapshot every 10K iterations -- ten times during training.
    s.snapshot = 10000
    s.snapshot_prefix = caffe_root + 'models/finetune_bird_recognition/bird_recognition'
    
    # Train on the GPU.  Using the CPU to train large networks is very slow.
    s.solver_mode = caffe_pb2.SolverParameter.GPU
    
    # Write the solver to a temporary file and return its filename.
    with tempfile.NamedTemporaryFile(delete=False) as f:
        f.write(str(s))
        return f.name

In [8]:
def run_solvers(niter, solvers, disp_interval=10):
    """Run solvers for niter iterations,
       returning the loss and accuracy recorded each iteration.
       `solvers` is a list of (name, solver) tuples."""
    blobs = ('loss', 'acc')
    loss, acc = ({name: np.zeros(niter) for name, _ in solvers}
                 for _ in blobs)
    for it in range(niter):
        for name, s in solvers:
            s.step(1)  # run a single SGD step in Caffe
            loss[name][it], acc[name][it] = (s.net.blobs[b].data.copy()
                                             for b in blobs)
        if it % disp_interval == 0 or it + 1 == niter:
            loss_disp = '; '.join('%s: loss=%.3f, acc=%2d%%' %
                                  (n, loss[n][it], np.round(100*acc[n][it]))
                                  for n, _ in solvers)
            print '%3d) %s' % (it, loss_disp)     
    # Save the learned weights from both nets.
    weight_dir = tempfile.mkdtemp()
    weights = {}
    for name, s in solvers:
        filename = 'weights.%s.caffemodel' % name
        weights[name] = os.path.join(weight_dir, filename)
        s.net.save(weights[name])
    return loss, acc, weights

In [9]:
niter = 1  # number of iterations to train

# Reset style_solver as before.
style_solver_filename = solver(style_net(train=True))

In [10]:
style_solver = caffe.get_solver(style_solver_filename)

In [11]:
style_solver.net.copy_from(weights)


In [12]:
scratch_style_solver_filename = solver(style_net(train=True))
scratch_style_solver = caffe.get_solver(scratch_style_solver_filename)

In [ ]:
print 'Running solvers for %d iterations...' % niter
solvers = [('pretrained', style_solver),
           ('scratch', scratch_style_solver)]
loss, acc, weights = run_solvers(niter, solvers)
print 'Done.'

train_loss, scratch_train_loss = loss['pretrained'], loss['scratch']
train_acc, scratch_train_acc = acc['pretrained'], acc['scratch']
style_weights, scratch_style_weights = weights['pretrained'], weights['scratch']

# Delete solvers to save memory.
del style_solver, scratch_style_solver, solvers

In [ ]:
niter = 1  # number of iterations to train

# Reset style_solver as before.
style_solver_filename = solver(style_net(train=True))
style_solver = caffe.get_solver(style_solver_filename)
style_solver.net.copy_from(weights)

# For reference, we also create a solver that isn't initialized from
# the pretrained ImageNet weights.
scratch_style_solver_filename = solver(style_net(train=True))
scratch_style_solver = caffe.get_solver(scratch_style_solver_filename)

print 'Running solvers for %d iterations...' % niter
solvers = [('pretrained', style_solver),
           ('scratch', scratch_style_solver)]
loss, acc, weights = run_solvers(niter, solvers)
print 'Done.'

train_loss, scratch_train_loss = loss['pretrained'], loss['scratch']
train_acc, scratch_train_acc = acc['pretrained'], acc['scratch']
style_weights, scratch_style_weights = weights['pretrained'], weights['scratch']

# Delete solvers to save memory.
del style_solver, scratch_style_solver, solvers

In [14]:
def eval_style_net(weights, test_iters=10):
    test_net = caffe.Net(style_net(train=False), weights, caffe.TEST)
    accuracy = 0
    for it in xrange(test_iters):
        accuracy += test_net.forward()['acc']
    accuracy /= test_iters
    return test_net, accuracy

In [ ]:
test_net, accuracy = eval_style_net(style_weights)
print 'Accuracy, trained from ImageNet initialization: %3.1f%%' % (100*accuracy, )
scratch_test_net, scratch_accuracy = eval_style_net(scratch_style_weights)
print 'Accuracy, trained from   random initialization: %3.1f%%' % (100*scratch_accuracy, )

In [ ]:
batch_index = 1
image = test_net.blobs['data'].data[batch_index]
plt.imshow(deprocess_net_image(image))
print 'actual label =', style_labels[int(test_net.blobs['label'].data[batch_index])]
disp_style_preds(test_net, image)

In [ ]:
def disp_preds(net, image, labels, k=5, name='ImageNet'):
    input_blob = net.blobs['data']
    net.blobs['data'].data[0, ...] = image
    probs = net.forward(start='conv1')['probs'][0]
    top_k = (-probs).argsort()[:k]
    print 'top %d predicted %s labels =' % (k, name)
    print '\n'.join('\t(%d) %5.2f%% %s' % (i+1, 100*probs[p], labels[p])
                    for i, p in enumerate(top_k))

def disp_imagenet_preds(net, image):
    disp_preds(net, image, imagenet_labels, name='ImageNet')

def disp_style_preds(net, image):
    disp_preds(net, image, style_labels, name='style')

In [ ]:
for image in images:
    net.blobs['data'].data[0, ...] = image
    probs = net.forward(start='conv1')['probs'][0]

In [ ]:
dummy_data = L.DummyData(shape=dict(dim=[1, 3, 227, 227]))  #图片数据转化好的
imagenet_net_filename = caffenet(data=dummy_data, train=False)
imagenet_net = caffe.Net(imagenet_net_filename, weights, caffe.TEST)  # 新weight

In [ ]:
import numpy as np
import pandas as pd
from pandas import  DataFrame
f = open("D:/biye/shuju/qqqq.txt")             # 返回一个文件对象  
line = f.readline()          # 调用文件的 readline()方法
#     计算概率
b=line.split('/')[-1].split(".")[0]
data = DataFrame(np.arange(90).reshape(30,3),columns=list('wxy'))
data['w']=b
data['x']=list(range(1,31))

for line in f:
#     计算概率    print (line)
    b=line.split('/')[-1].split(".")[0]
    data1 = DataFrame(np.arange(90).reshape(30,3),columns=list('wxy'))

    data1['w']=b
    data1['x']=list(range(1,31))
    data=pd.concat([data,data1])

f.close()

In [ ]:
    blobs = ('loss', 'acc')
    loss, acc = ({name: np.zeros(niter) for name, _ in solvers}
                 for _ in blobs)

In [11]:
blobs = ('loss', 'acc')

In [12]:
blobs

('loss', 'acc')

In [28]:
solvers=[('name1',5),('name', 5)]


In [29]:
solvers

[('name1', 5), ('name', 5)]

In [30]:
blobs = ('loss', 'acc')
loss, acc = ({name: np.zeros(10) for name, _ in solvers}
                 for _ in blobs)

In [31]:
{name: np.zeros(10) for name, _ in solvers}

{'name': array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 'name1': array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])}

In [24]:
loss[('name1',
  'solver1')]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [12]:
acc

{('name1',
  'solver1'): array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])}